### Stream Customers Data From CloudFiles to Delta lake
- 1.Read files from cloud storage using DataStreamReader API
- 2.Transform the dataframe to add the following columns
-         file paath : Cloud File path
-         ingestion date :current Timestamp
- 3.Write the transformed data stream to Delta Lake Table

#### 1.Read files from cloud storage using DataStreamReader API

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType,DateType

customers_schema = StructType(fields =[StructField("customer_id",IntegerType()),
                                       StructField("customer_name",StringType()),
                                       StructField("date_of_birth",DateType()),
                                       StructField("telephone",StringType()),
                                       StructField("email",StringType()),
                                       StructField("member_since",StringType()),
                                       StructField("created_timestamp",TimestampType())])

In [0]:
df_cus_stream_nw = spark.readStream.format('json').schema(customers_schema).load('/Volumes/gizmobox/landing/operational_data/customers_stream')

2.Transform the dataframe to add the following columns
-     file paath : Cloud File path
-     ingestion date :current Timestamp

In [0]:
from pyspark.sql.functions import *
df_transformed = (df_cus_stream_nw.

                withColumn("file_path",col("_metadata.file_path")).
                withColumn("Ingestion_date",current_timestamp())
)

3.Write the transformed data stream to Delta Lake Table

In [0]:
df_transformed.writeStream.format("delta").option("checkpointLocation", "/Volumes/gizmobox/landing/operational_data/customers_stream/_checkpoint_stream_nw").toTable("gizmobox.bronze.customers_stream_nw")


In [0]:
%sql
select * from gizmobox.bronze.customers_stream_nw;

In [0]:
df_transformed.writeStream.format("delta").outputMode("append").trigger(processingTime="2 minutes").option("checkpointLocation", "/Volumes/gizmobox/landing/operational_data/customers_stream/_checkpoint_stream_nw").toTable("gizmobox.bronze.customers_stream_nw")